## Chapter 3: When Objects Are Alike
By Will Norris


#### Key Concepts: 
- We want to limit repeated code!
- When we have similar objects, we can relate them to eachother to avoid repeating ourselves 
    - Inheritance!

Inheritence gives us the power to do __hierarchical clustering__

![](inheritence_animals.gif)

**Inheritence defines an 'is a' relationship**

### Extending built-ins!
#### We can build off of python's already created objects using inheritence!

In [12]:
class AnimalList(list): # inherits from list object 
    def search(self, name):
        matches = []
        for animal in self: 
            if name in animal.name:
                matches.append(animal)
        return matches

- This allows us to add functionality to ```list``` without having to make the actual ```list``` again!

### Overriding

In [67]:
class Animal:
    all_animals = AnimalList()
    def __init__(self, name):
        self.name = name 
        self.all_animals.append(self)

class Mammal(Animal):
    live_born = True # live birth is an attribute of mammals
    def __init__(self, name, land_water): 
        self.name = name 
        self.land_water = land_water        

In [68]:
m1 = Mammal('dolphin', 'water')
m2 = Mammal('dog', 'land')
a1 = Animal('shrimp')

for animal in Animal.all_animals:
    print(animal.name)

shrimp


- Here, we do succesfully create a Mammal, but we don't add mammals to our ```all_animals``` list!!
    - What's going wrong? 

### Super to the rescue!
- ```super``` allows the class to call the parent's constructor (pass info up the hierarchy)

In [15]:
class Mammal(Animal):
    live_born = True # live birth is an attribute of all mammals
    def __init__(self, name, land_water):
        super().__init__(name)
        self.land_water = land_water
    
    def talk(self):
        print("Most Mammal's can make a noise of some kind.")

In [16]:
m1 = Mammal('Bob the dolphin', 'water')
m2 = Mammal('Frank the Dog', 'land')
a1 = Animal('Josie the Shrimp')

for animal in Animal.all_animals:
    print(animal.name)

shrimp
Bob the dolphin
Frank the Dog
Josie the Shrimp


## Fun with Dunders
### \__repr__ and \__str__: Letting users easily see what their objects are 
- \__repr__ is to find the "official" representation of an object 
- \__str__ is to find the "informal" representation of an object


- If you only have one of them, calling the object will use whichever is available
- If you have both of them, calling the class will default to the \__str__ representation 

In [17]:
class Dog(Mammal):
    fur = True 
    def __init__(self, name, breed, age):
        super().__init__(name, land_water = 'land')
        self.breed = breed
        self.age = age
        
    def __repr__(self):
        return "Animal of class {}, type of {}, breed of {}, age of {}".format(
                self.__class__.__bases__[0].__name__,self.__class__.__name__, self.breed, self.age)
    
    def __str__(self):
        return "This dog's name is {}".format(self.name)
    
    def talk(self): 
        print("{} Barked at you!".format(self.name))

class Whale(Mammal):
    fur = False
    def __init__(self, name, species, age): 
        super().__init__(name, land_water = 'water')
        self.species = species 
        self.age = age
        
    def __repr__(self):
        return "Animal of class {}, type of {}, species of {}, age of {}".format(
                self.__class__.__bases__[0].__name__,self.__class__.__name__, self.species, self.age)

    def __str__(self):
        return "This {}'s name is {}".format(self.__class__.__name__,self.name)
    
    def talk(self):
        print("{} makes a loud groaning noise".format(self.name))

In [18]:
frank = Dog('frank', 'Golden Retreiver', 10)
waler = Whale('waler', 'Blue', 80)

# defualt to __str__ representation (informal)
print(frank)
print(waler)

print(repr(frank))
# or 
print(waler.__repr__())

This dog's name is frank
This Whale's name is waler
Animal of class Mammal, type of Dog, breed of Golden Retreiver, age of 10
Animal of class Mammal, type of Whale, species of Blue, age of 80


### Multiple Inheritence: Warning Danger Ahead 



![MI](mi.png)

Why multiple inheritence isn't best practice: 
- It is the opposite structure of **hierarchical clustering**
- M.I. seems okay when we are mixing methods from different classes
- However, it gets messy when we call methods on the superclass 
    - There's more than one superclass! Which one do we call, in what order?? 
    - Example coming up!

![](multi_pegasus.png)

- This looks like it should work, a pegasus is a horse with wings, BUT:
    - Birds have other traits Pegasi don't
        - **We can't selectively inherit our traits!!**
        - If pegasi have live birth, but birds don't, then we need to restructure
- There are other ways to achieve a similar goal, which we will cover next!

### Making Multiple Inheritence "Work": Super Saves Us Again

In [78]:
# borrowed from our book :-)
class Contact:
    all_contacts = []

    def __init__(self, name=None, email=None, street = None, **kwargs):
        super().__init__(**kwargs)
        self.name = name
        self.email = email
        self.street = street*2
        self.all_contacts.append(self)


class AddressHolder:
    def __init__(self, street=None, city=None, state=None, code=None, **kwargs):
        super().__init__(**kwargs)
        self.street = street
        self.city = city
        self.state = state
        self.code = code


class Friend(Contact,AddressHolder): # Friend inherits from Contact and AdressHolder
    def __init__(self, phone='', **kwargs):
        super().__init__(**kwargs)
        self.phone = phone

In [79]:
kwargs = {'city':'boulder','name':'will'}
f = Friend('719-220-3333', **kwargs)
print(f.street)
print(f.name)

TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

- Super works a little weird when there is more than one parent class 
- Python uses a linearized method resolution order
    - If we instanitate ```Friend```, it will call the ```__init__``` from ```Contact``` then the ```__init__``` in ```AddressHolder```
- We use ```**kwargs``` to pass potentially "extra" arguments to an ```__init__``` in case future ```__init__```'s need it later
    - This gets messy, and requires docstrings to instruct users on what is expected on instantiation 

### The Diamond Problem

![](diamond_prob.png)

- The Object parent class is instantiated twice (by Contact and AddressHolder)
   - This is okay right now, but it can really wreak havoc when the parent class isn't ```Object```

- While we can make multi-inheritence work, it has many downsides to organization
    - Can require complete restructuring of classes and there parameters 
        - Must be planned for from the beggining!
    - There are better solutions! Namely **composition**, covered in chapter 10

### Polymorphism 

- Allow user to call the same method name with parameters, and depending on those parameters, it can do different things!
- Polymorphism with inheritance, under the hood: 
    - subclass can override a method of the base class 
    - Class ```Mammal``` has ```talk()``` method:
        - subclasses ```Dog``` and ```Whale``` have the method ```talk()``` make different noises 

In [26]:
# simple example without inheritence 
print(5*6)
print(5*'hello')

30
hellohellohellohellohello


In [27]:
m1.talk()
frank.talk()
waler.talk()

Most Mammal's can make a noise of some kind.
frank Barked at you!
waler makes a loud groaning noise


#### Polymorphism is cool, but Python makes it less cool with "Duck Typing"
- "Duck typing in Python allows us to use any object that provides the requird behavior without forcing it to be a subclass" (pg. 77) 

- Polymorphism combined with inheritance is powerful, however it requires lot's of planning and can get complex fast!
    - Especially when you are working with lots of classes 


### Duck Typing 
- "If it walks like a duck and it quacks like a duck, then it must be a duck" 

- We use this idea to help us **determine if an object can be used for a specific purpose**
    - Rather than just checking the type of the object, we check what attributes and methods that it has 
    
- Duck Typing removes the need to inherit in certain cases, and can simplify your life! 
    
- **Easier to Ask Forgiveness than Permission (EAFP)**
    - Duck typing is an introduction to this best practice of python programming!


In [28]:
class Duck: 
    def fly(self): 
        print("Duck Flying")
        
    def quack(self):
        print("quack, quack, quack")
        
class Airplane:
    def fly(self):
        print("Airplane Flying")
        
    def quack(self):
        print("Pilot imitates a duck, 'Quack, Quack'")

class Whale:
    def swim(self):
        print("Whale swimming")
        
def lift_off(entity):
    entity.fly()

In [29]:
duck = Duck()
airplane = Airplane()
whale = Whale()

lift_off(duck)
lift_off(airplane)
lift_off(whale)

Duck Flying
Airplane Flying


AttributeError: 'Whale' object has no attribute 'fly'

#### The old way: Look Before you Leap (LBYL)
- check something is what we want before trying to do it 

In [30]:
def quack_and_fly(my_object):
    if hasattr(my_object, 'quack'):
        if callable(my_object.quack):
            my_object.quack()
    if hasattr(my_object, 'fly'):   
        if callable(my_object.fly):
            my_object.fly()

d = Duck()
quack_and_fly(d)

a = Airplane()
quack_and_fly(a)

w = Whale()
quack_and_fly(w)

quack, quack, quack
Duck Flying
Pilot imitates a duck, 'Quack, Quack'
Airplane Flying


BUT, most of the time our object will work, which means we are doing way more checks than we need to! And in this case, nothing happens if an object doesn't fit.

### EAFP
- Try to do something, if it works, then great! If not, then let's handle it 

In [33]:
def lift_off(entity):
    try: 
        entity.fly()
        entity.quack()
    except Exception as e: 
        print(e)

In [36]:
lift_off(duck)
lift_off(airplane)
lift_off(whale)

Duck Flying
quack, quack, quack
Airplane Flying
Pilot imitates a duck, 'Quack, Quack'
'Whale' object has no attribute 'fly'


### EAFP is useful outside of object oriented programming too!

In [39]:
animal1 = {'name': 'Frank', 'age':10, 'type':'dog', 'pronoun':'He'}
animal2 = {'name': 'Josie', 'age':5, 'type':'cat'}

def animal_deets(animal):
    try:
        print("This animal is a {type}, named {name}. {pronoun} is {age} years old".format(**animal))
    except Exception as e:
        print("Missing {} key".format(e))

In [40]:
animal_deets(animal1)

animal_deets(animal2)

This animal is a dog, named Frank. He is 10 years old
Missing 'pronoun' key


### Abstract Base Classes (ABC's) 
- Duck typing is great! But, it can get complicated with many classes 
- **ABC's define a set of methods and properties that a class must implement in order to be considered a duck-type instance of that class**
    - Basically just ensures that certain methods are available in order to be considered a duck typed instance of a class 



In [43]:

from collections import Container 

Container.__abstractmethods__

frozenset({'__contains__'})

In [44]:
class OddContainer:
    def __contains__(self, x):
        if not isinstance(x, int) or not x % 2: 
            return False
        return True

In [45]:
odd_container = OddContainer()
print(isinstance(odd_container, Container))
print(issubclass(OddContainer, Container))

True
True


- This is the power of duck typing! 
    - Even though ```OddContainer``` is not inheriting from Container, it still is considered a subclass and instance of the Container class. 
    - We get all the benifits of polymorphism without dealing with the mess of inheritance!

#### ABC's Under the Hood 

In [60]:
from abc import ABC, abstractmethod 

class Shape(ABC):
    @abstractmethod 
    def __area__(self): pass
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Shape:
            if any("__area__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented
    
class Square():
    def __init__(self, side_len):
        self.side = side_len 
    
    def __area__(self):
        return self.side * self.side

In [61]:
s = Square(5)

In [62]:
print(issubclass(Square, Shape))
print(isinstance(s, Shape))

True
True


In [65]:
from abc import ABC, abstractmethod 

class Shape(ABC):
    @abstractmethod 
    def __area__(self): pass
    
    
class Square(Shape):
    def __init__(self, side_len):
        self.side = side_len 
    
    def __area__(self):
        return self.side * self.side

In [66]:
s = Square(5)